In [ ]:
N = ord("M") % 5 + 1
print(N)

In [ ]:
import pandas as pd
import cv2
import numpy as np
from PIL import Image, ImageDraw, ImageColor
import os
import math
import matplotlib.pyplot as plt

df = pd.read_excel('kr1.xlsx')

# correct color names
def validate_color(color):
    color = color.lower()
    color_map = {
        'cian': 'cyan',
        'magenta': 'magenta',
        'yellow': 'yellow',
        'red': 'red',
        'blue': 'blue'
    }
    return color_map.get(color, color)

# Function to add round glasses and improve eye detection
def add_round_glasses(image_path, output_size, glasses_color, line_width):
    # Construct the full path to the image
    full_image_path = os.path.join("Images", image_path)
    
    # Check if the file exists
    if not os.path.exists(full_image_path):
        print(f"Error: The file {full_image_path} does not exist.")
        return None, None
    
    # Read the image
    img = cv2.imread(full_image_path)
    
    # Check if the image was successfully loaded
    if img is None:
        print(f"Error: Unable to read the image {full_image_path}")
        return None, None
    
    # Store the original image
    original_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Load the face cascade
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Adjust the parameters of face detection for better accuracy
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    
    if len(faces) == 0:
        print("No face detected")
        return original_img, None
    
    # Get the first face
    (x, y, w, h) = faces[0]
    
    # Load the eye cascade
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    
    # Detect eyes in the upper half of the face
    roi_gray = gray[y:y + h//2, x:x + w]  # Use upper half of the face for better accuracy
    eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10, minSize=(10, 10))
    
    if len(eyes) < 2:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=10, minSize=(10, 10))
    
    if len(eyes) < 2:
        print("Not enough eyes detected")
        return original_img, None
    
    # Sort eyes by x-coordinate to ensure left eye is first
    eyes = sorted(eyes, key=lambda e: e[0])
    
    # Get the first two eyes
    left_eye, right_eye = eyes[:2]
    
    # Calculate eye centers
    left_eye_center = (x + left_eye[0] + left_eye[2]//2, y + left_eye[1] + left_eye[3]//2)
    right_eye_center = (x + right_eye[0] + right_eye[2]//2, y + right_eye[1] + right_eye[3]//2)
    
    # Calculate distance between eyes
    eye_distance = np.sqrt((right_eye_center[0] - left_eye_center[0])**2 + 
                           (right_eye_center[1] - right_eye_center[1])**2)
    
    # Calculate lens radius based on eye distance
    lens_radius = int(eye_distance * 0.35)
    
    # Create a new image with PIL
    pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    
    # Validate and correct color name
    glasses_color = validate_color(glasses_color)
    
    try:
        # Draw left lens
        draw.ellipse([left_eye_center[0] - lens_radius, 
                      left_eye_center[1] - lens_radius,
                      left_eye_center[0] + lens_radius, 
                      left_eye_center[1] + lens_radius],
                     outline=glasses_color, width=line_width)
        
        # Draw right lens
        draw.ellipse([right_eye_center[0] - lens_radius, 
                      right_eye_center[1] - lens_radius,
                      right_eye_center[0] + lens_radius, 
                      right_eye_center[1] + lens_radius],
                     outline=glasses_color, width=line_width)
        
        # Draw bridge
        bridge_y = (left_eye_center[1] + right_eye_center[1]) // 2
        draw.line([left_eye_center[0] + lens_radius, bridge_y,
                   right_eye_center[0] - lens_radius, bridge_y],
                  fill=glasses_color, width=line_width)
        
        # Draw small red circles at the eye centers for visualization
        # eye_marker_radius = 5
        # draw.ellipse([left_eye_center[0] - eye_marker_radius, left_eye_center[1] - eye_marker_radius,
        #               left_eye_center[0] + eye_marker_radius, left_eye_center[1] + eye_marker_radius],
        #              outline="red", fill="red")
        # draw.ellipse([right_eye_center[0] - eye_marker_radius, right_eye_center[1] - eye_marker_radius,
        #               right_eye_center[0] + eye_marker_radius, right_eye_center[1] + eye_marker_radius],
        #              outline="red", fill="red")

        # Calculate temple angle and length
        temple_angle = math.atan2(right_eye_center[1] - left_eye_center[1],
                                  right_eye_center[0] - left_eye_center[0])
        temple_length = int(eye_distance * 0.8) // 3
        
        # Left temple
        left_temple_end = (
            int(left_eye_center[0] - lens_radius - temple_length * math.cos(temple_angle + math.pi/6)),
            int(left_eye_center[1] - temple_length * math.sin(temple_angle + math.pi/6))
        )
        draw.line([left_eye_center[0] - lens_radius, left_eye_center[1],
                   left_temple_end[0], left_temple_end[1]],
                  fill=glasses_color, width=line_width)
        
        # Right temple
        right_temple_end = (
            int(right_eye_center[0] + lens_radius + temple_length * math.cos(temple_angle - math.pi/6)),
            int(right_eye_center[1] + temple_length * math.sin(temple_angle - math.pi/6))
        )
        draw.line([right_eye_center[0] + lens_radius, right_eye_center[1],
                   right_temple_end[0], right_temple_end[1]],
                  fill=glasses_color, width=line_width)
        
    except ValueError as e:
        print(f"Error: Invalid color '{glasses_color}'. {str(e)}")
        return original_img, None
    
    # Resize the images
    original_img = original_img.resize((output_size, output_size))
    pil_img = pil_img.resize((output_size, output_size))
    
    # Save the result
    output_path = f"output_{os.path.basename(image_path)}"
    pil_img.save(output_path)
    print(f"Processed image saved as {output_path}")
    
    return original_img, pil_img



# Process the specific row
N = ord("E") % 5 + 1
print(f"Processing row {N}")

row = df.iloc[N - 1]  # Subtract 1 because DataFrame indexing is 0-based
image_path = row['file name']
output_size = int(row['image size'].split('x')[0])  # Assuming square output
glasses_color = row['glasses color']
line_width = row['line width']

# Call the function
original_img, result_img = add_round_glasses(image_path, output_size, glasses_color, line_width)

if original_img is not None and result_img is not None:
    print("Image processing completed successfully.")
    
    # Create a figure with two subplots side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    
    # Display original image
    ax1.imshow(original_img)
    ax1.set_title('Original Image')
    ax1.axis('off')
    
    # Display processed image
    ax2.imshow(result_img)
    ax2.set_title('Processed Image')
    ax2.axis('off')
    
    # Adjust the layout and display the plot
    plt.tight_layout()
    plt.show()
    
elif original_img is not None:
    print("Image processing failed. Displaying original image only.")
    
    plt.figure(figsize=(6, 6))
    plt.imshow(original_img)
    plt.title('Original Image')
    plt.axis('off')
    plt.show()
    
else:
    print("Image processing failed. No images to display.")